In [1]:
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
from PyQt5 import *
import sys

In [2]:
class Window(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('DCiFR')
        self.setGeometry(0, 0, 600, 400)
        
        #Page title/main label
        self.title_label = QLabel('Welcome to DCiFR!', self)
        self.title_label.move(100, 50)
        self.title_label.setFont(QFont('Arial', 20))
        self.title_label.adjustSize()
        
        #hover info 
        info = QLabel('Check the box that applies. Hover for more info!', self)
        info.move(50, 125)
        info.setFont(QFont('Arial', 10))
        info.adjustSize()
        myFont=QtGui.QFont()
        myFont.setItalic(True)
        info.setFont(myFont)
        info.adjustSize()
        
        #Check boxes
        single_cb = QCheckBox('Single Image', self)
        single_cb.move(50, 150)
        batch_cb = QCheckBox('Batch Mode', self)
        batch_cb.move(50, 200)
        single_cb.adjustSize()
        batch_cb.adjustSize()
        
        #Hovers
        single_cb.setToolTip('Check this box if you would like to analyze demograhpics for a single image')
        batch_cb.setToolTip('Check this box if you would like to analyze demograhpics for more than one image')
        
        #next button
        next_button = QPushButton('Next', self)
        next_button.setToolTip('Click this button after you have made your selection')
        next_button.move(450, 350)

app = QApplication(sys.argv)
window = Window()
window.show() 
app.exec_()

0